In [1]:
!pip install openai requests pydub

In [ ]:
import os
import openai
import datetime
import requests
from google.colab import files
from requests.exceptions import RequestException

In [2]:
def download_mp3(url, file_path):
    try:
        # Send a HTTP request to the URL of the MP3 file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors
        # Open the file in write mode to download the MP3 file
        with open(file_path, 'wb') as audio:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive new chunks
                    audio.write(chunk)
        print(f"Downloaded successfully: {file_path}")
        return file_path
    except RequestException as e:
        print(f"Error during the request of {url}: {str(e)}")
        return None
    except Exception as e:
        print(f"Error during the download of {url}: {str(e)}")
        return None

In [ ]:
# Transcribe audio file using Whisper API
def transcribe_audio_whisper_api(file_path):
    try:
        with open(file_path, "rb") as file:
            transcription = openai.Audio.transcribe("whisper-1", file)
        return transcription
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in transcription: {e}")
        return None


In [ ]:
# Analyze emotion from text using OpenAI GPT model
def analyze_emotion(text):
    try:
        content = f"What emotion is the following text expressing?\n{text}"
        messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": content}]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=200
        )
        emotion = response['choices'][0]['message']['content']
        return emotion.strip()
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in emotion analysis: {e}")
        return None


In [ ]:
# Summarize text using OpenAI Chat model
def summarize_text(text, style):
    try:
        # Using the chat endpoint
        if style == "1":
            content = f"Summarize the following text in bullet points, ordered by importance:\n{text}"
        elif style == "2":
            content = f"Summarize the following text in a short paragraph of 4 to 5 lines:\n{text}"
        else:
            content = f"Summarize the following text concisely:\n{text}"

        messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": content}]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=500
        )
        summary = response['choices'][0]['message']['content']
        return summary
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in summarization: {e}")
        return None


In [ ]:
# User input for source of MP3 file
print("Select the source of MP3 file:")
print("1 - Upload File")
print("2 - Insert MP3 Link")
source = input("Enter 1 or 2: ")

file_path = None
if source == "1":
    # Upload the MP3 file
    uploaded = files.upload()
    for mp3_filename in uploaded.keys():
        print(f"{datetime.datetime.now()} [INFO]: User uploaded file '{mp3_filename}'")
        file_path = mp3_filename
elif source == "2":
    # Download the MP3 file from the given link
    url = input("Enter the MP3 link: ")
    file_path = "audio.mp3"  # you can set to any path you want
    file_path = download_mp3(url, file_path)
else:
    print("Invalid option selected. Please select either 1 or 2.")

if file_path:
    # Transcribe the audio using Whisper API
    print(f"{datetime.datetime.now()} [INFO]: Transcribing audio...")
    transcription = transcribe_audio_whisper_api(file_path)

In [ ]:
  # User input for summarization style
    print("Select summarization style:")
    print("1 - Bullet Points")
    print("2 - Short Paragraph")
    print("3 - Concise")
    style = input("Enter 1, 2, or 3: ")

In [ ]:
  # Analyze emotion from the transcription using OpenAI GPT model
    if transcription:
        print(f"{datetime.datetime.now()} [INFO]: Analyzing emotion...")
        emotion = analyze_emotion(transcription)
        print(f"{datetime.datetime.now()} [INFO]: Emotion: {emotion}")

        # Summarize the transcription using OpenAI Chat model
        print(f"{datetime.datetime.now()} [INFO]: Summarizing text...")
        summary = summarize_text(transcription, style)
        print(f"{datetime.datetime.now()} [INFO]: Summary: {summary}")

        # Download the summary as a text file
        with open('summary.txt', 'w') as f:
            f.write(f"Emotion: {emotion}\n\n")
            f.write(summary)
        files.download('summary.txt')
